In [1]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [3]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [5]:
features = [
    'wait_time', 
    'delay_vs_expected', 
    'distance_seller_customer',
    'price',
    'freight_value'
]

🕵🏻 Check the `multicollinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [7]:
# imprt. scaler to stndze feat. & rmv scale effct 
from sklearn.preprocessing import StandardScaler
# xtrct selcted feat. & drp misng vals
X = orders[features].dropna()
# int sclr.
scaler = StandardScaler()
# fit sclr & trnsfrm data
X_scaled = scaler.fit_transform(X)
# cnvrt scld. data bk to df
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=features)

👉 Run your VIF Analysis to analyze the potential multicollinearities:

In [8]:
# Calc VIF for ea feat to chk for mlticol.
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Prep df to stor VIF val's
vif_data = pd.DataFrame()
vif_data['feature'] = X_scaled.columns

# Calc VIF for ea feat
vif_data['VIF'] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Disp VIF val's
print(vif_data)

                    feature       VIF
0                 wait_time  2.614684
1         delay_vs_expected  2.211188
2  distance_seller_customer  1.412197
3                     price  1.206315
4             freight_value  1.335396


In [ ]:
!git add -A
!git commit -m "Decision_Science-Logistic_Regression-Logit-1"

## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [9]:
# Get targt var's, align w X_scaled
y_one = orders.loc[X_scaled.index, 'dim_is_one_star']

# Add const term to predctrs
X_const = sm.add_constant(X_scaled)

# Fit 2 log reg modl
logit_one = sm.Logit(y_one, X_const).fit()

# Prnt sumry:
print(logit_one.summary())

Optimization terminated successfully.
         Current function value: 0.280634
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95866
Method:                           MLE   Df Model:                            5
Date:                Thu, 13 Feb 2025   Pseudo R-squ.:                  0.1227
Time:                        13:16:37   Log-Likelihood:                -26905.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -2.4179      0.013   -193.169      0.000      -2.442

`Logit 5️⃣`

In [ ]:
# YOUR CODE HERE

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [ ]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = []

🧪 __Test your code__

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())

<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !